In [1]:
import cv2
import dlib
import pickle
import numpy as np
import math
from sklearn.svm import SVC

In [2]:
data = {} 
data['landmarks_vectorised'] = [] 
emotions = ["colere", "degout", "peur", "joie", "tristesse", "surprise", "neutre"] #Initialisation du Dictionnaire de Données

In [3]:
def get_landmarks(image):
    # Détecter les visages dans l'image
    detections = detector(image, 1)
    
    # Pour chaque instance de visage détectée individuellement
    for k, d in enumerate(detections):
        # Dessiner les landmarks faciaux avec la classe predictor
        shape = predictor(image, d)

        # Initialiser les listes pour stocker les coordonnées X et Y
        xlist = []
        ylist = []

        # Stocker les coordonnées X et Y dans les listes respectives
        for i in range(1, 68):
            xlist.append(float(shape.part(i).x))
            ylist.append(float(shape.part(i).y))

        # Calculer les valeurs moyennes des coordonnées X et Y
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)

        # Calculer les déviations centrales par rapport aux moyennes
        xcentral = [(x - xmean) for x in xlist]
        ycentral = [(y - ymean) for y in ylist]

        # Initialiser la liste pour stocker les landmarks normalisés
        landmarks_vectorised = []

        # Analyser la présence de landmarks faciaux
        for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
            landmarks_vectorised.append(w)
            landmarks_vectorised.append(z)

            # Extraire le centre de gravité avec la moyenne des axes
            meannp = np.asarray((ymean, xmean))
            coornp = np.asarray((z, w))

            # Mesurer la distance et l'angle de chaque landmark par rapport au centre de gravité
            dist = np.linalg.norm(coornp - meannp)
            landmarks_vectorised.append(dist)
            landmarks_vectorised.append((math.atan2(y, x) * 360) / (2 * math.pi))
        
        # Stocker les landmarks dans le dictionnaire global
        data['landmarks_vectorised'] = landmarks_vectorised
    
    # Si aucun visage n'est détecté, stocker une erreur dans le dictionnaire
    if len(detections) < 1:
        data['landmarks_vestorised'] = "error"


In [4]:
# Configuration des objets requis
pkl_filename = 'pickle_model_2.pkl'  # Fichier du modèle entraîné
with open(pkl_filename, 'rb') as file:  # Charger tous les poids du modèle
    pickle_model = pickle.load(file)

# Capture de la webcam
video_capture = cv2.VideoCapture(0)

# Initialisation du détecteur de visages
detector = dlib.get_frontal_face_detector()

# Initialisation du prédicteur de landmarks
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Taille du cadre de capture (ajustable)
FACE_SHAPE = (200, 200)


In [5]:
while True:
    ret, frame = video_capture.read()

    # Redimensionner le cadre à une taille plus petite pour un traitement plus rapide
    frame = cv2.resize(frame, FACE_SHAPE)

    # Convertir l'image en niveaux de gris (notre ensemble de données est en niveaux de gris)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Appliquer une égalisation adaptative de l'histogramme local (CLAHE) pour améliorer le contraste
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_image = clahe.apply(gray)
    
    # Obtenir les landmarks à partir de l'image traitée
    get_landmarks(clahe_image)

    # Si des landmarks sont détectés
    if data['landmarks_vectorised'] != "error":
        # Convertir les landmarks en tableau numpy
        prediction_data = np.array(data['landmarks_vectorised'])

        # Prédire les étiquettes émotionnelles
        predicted_labels = pickle_model.predict(prediction_data.reshape(1, -1))

        # Afficher l'émotion prédite sur la vidéo en rouge
        emotion_text = emotions[predicted_labels[0]]
            
        if emotion_text != "neutre":
            cv2.putText(frame, f"Emotion: {emotion_text}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    else:
        # Afficher un message si aucun visage n'est détecté
        cv2.putText(frame, "Aucun visage détecté.", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Afficher la sortie de la webcam
    cv2.imshow("image", frame)

    # Sortir de la boucle si l'utilisateur appuie sur 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        video_capture.release()
        cv2.destroyAllWindows()
        break
